In [16]:
#%%writefile dbpdf.py
import getpass
import pymysql
import unidecode
import pandas as pd
import tempfile
import subprocess
import sys

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth',500)

def generate_pdf(row):
    
    latexfile=get_latexfile(row)

    f=tempfile.NamedTemporaryFile('w',suffix='.tex',delete=False)
    f.write(latexfile)
    f.close()
    lo=subprocess.Popen('pdflatex {:s}'.format(f.name).split(),cwd='/tmp',
                        stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()
    pdffile='{:s}'.format(f.name).split('.tex')[0]+'.pdf'
    if lo[0].decode('utf8').find(pdffile.split('/')[-1])>-1:
        pdf='carta_{:d}.pdf'.format(row['id'])
        dwld_dir='downloads'
        kk=subprocess.Popen('mv {:s} /home/restrepo/{:s}/{:s}'.format(pdffile,dwld_dir,pdf).split()).communicate()
        print('output in http://fisica.udea.edu.co/{:s}/{:s}'.format(dwld_dir,pdf))
        for suffix in  ['.aux','.log','.out','.tex']:
            sc=subprocess.Popen( ['rm', '{:s}'.format(f.name).split('.tex')[0]+suffix] ).communicate()
    else:
        print(lo[0].decode('utf8'))
        sys.exit('LaTeX ERROR:')    
    return latexfile

def db_to_pdf(choose_db='Diego_Restrepo',id=None,aux_tables='cartas_sign',common_columns='signature',verbose=False):
    '''
    Obtain LaTeX file by using the data from Data Base choose_db with tables defined
    (in) inside (TODO: configuration file) with auxiliar table aux_tables
    Type of letter: 
      * 0 Personal
      * 1 Group
    '''
    password=getpass.getpass()
    #TODO: read from config
    tipo_carta={'Diego_Restrepo':'cartas', 'cartas_gfif':'cartas','seminarios':'talks'}
    conn = pymysql.connect(host='localhost', port=3306, user='root', passwd=password, db=choose_db)

    df=pd.read_sql('select * from {};'.format(tipo_carta[choose_db]), con=conn)

    #TODO: generilize to addtional tables
    if aux_tables:
        sg=pd.read_sql('select * from {};'.format(aux_tables), con=conn)
    
    
    conn.close()
    
    #print(df.shape[0])
    #return df,sg
    if df.shape[0]:
        if not id:
            id=df.id.values[-1]
        c=df[df.id==id].reset_index(drop=True).loc[0]
        if aux_tables:
            #TODO:
            c['common_{}'.format(common_columns)]=sg[
                sg[common_columns]==c[common_columns]].reset_index(drop=True).loc[0,'sign']
        if verbose:
            print(c)
    else:
        sys.exit('Error reading databases')
        return df

    
    
    return generate_pdf(c)

def backup_database(choose_db=0):
    '''
    Obtain letter from the last DB entry
    Type of letter: 
      * 0 Personal
      * 1 Group
    '''
    password=getpass.getpass()
    tipo_carta=['Diego_Restrepo', 'cartas_gfif']
    
    print('Trying to create backup file db_{:s}_backup.sql'.format( tipo_carta[choose_db]  )   )
    ok=subprocess.Popen('mysqldump -u root -p{:s} {:s} > db_{:s}_backup.sql'.format(
                     password,tipo_carta[choose_db],tipo_carta[choose_db]),shell=True,
                     stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()
    
    print()
    
    return ok

In [17]:
db_to_pdf(choose_db='seminarios',aux_tables='',id=302)

········
output in http://fisica.udea.edu.co/downloads/carta_302.pdf


'\n\\documentclass[xcolor=pdftex,dvipsnames,table]{beamer}\n\\pdfoutput=1\n\\mode<presentation>\n{\n  \\setbeamercovered{transparent}\n  \\setbeamertemplate{blocks}[rounded][shadow=true]\n  \\setbeamertemplate{navigation symbols}{}\n  \\setbeamertemplate{footline}{%\\hspace*{.5cm}\n    \\scriptsize{\\phantom{Gg}%\\insertauthor\n      \\hspace*{50pt}\n      \\hfill \\insertframenumber\n      \\hspace*{.5cm}}}\n}\n\\usepackage[T1]{fontenc}\n\\usepackage[utf8]{inputenc}\n\\usepackage[spanish]{babel}\n\\spanishdecimal{.}\n\\usepackage{beamerprosper}\n\\usepackage{amsmath,amssymb}\n\\usepackage{graphicx}\n\\usepackage{mycolors}\n\\usepackage{pgf}\n\\setlength{\\textwidth}{120 mm}\n\\setbeamersize{text margin left=10pt,text margin right=10pt}\n\\newcommand{\\widescreen}{\n\\setlength{\\paperwidth}{171 mm}\n\\setlength{\\paperheight}{96 mm}\n\\setlength{\\textwidth}{161 mm}\n\\setlength{\\textheight}{86 mm}\n}\n\\begin{document}\n\\setbeamertemplate{background}{\\includegraphics[width=\\paper